BIOS 512
HOMEWORK 8
BELLA WILLIAMS

QUESTION 1

A) Given a number of clusters to search for N:

1. Assign each point to a cluster N at random.
2. Calculate the mean position of each cluster using the previous assignments.
3. Loop through the points - assign each point to the cluster to whose center it is closest.
4. Repeat this process until the centers stop moving around.

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
library(ggplot2)

In [ ]:
library(dplyr)

In [ ]:
library(readr)

B) Step 1

In [ ]:
label_randomly <- function(n_points, n_clusters){
sample(1:n_clusters, size = n_points, replace = TRUE)
}

C) Step 2

In [2]:
get_cluster_means <- function(data, labels){
  library(dplyr)
  
  data %>%
    mutate(label__ = labels) %>%
    group_by(label__) %>%       

    summarize(across(
      .cols = where(is.numeric), 
      .fns = mean, 
      .names = "{.col}"
    )) %>%
    
    rename(label = label__) %>%
    ungroup() %>%
    arrange(label)
}

D) Step 3

In [3]:
assign_cluster <- function(data, means){

  X <- as.matrix(data) 
  C <- as.matrix(means %>% select(-label)) 
  
  X_sq <- rowSums(X * X)
  C_sq <- rowSums(C * C)
  

  dist_sq_matrix <- outer(X_sq, C_sq, FUN = "+") - 2 * (X %*% t(C))
  

  new_labels <- max.col(-dist_sq_matrix)
  
  return(new_labels)
}

E) Step 4

In [4]:
kmeans_done <- function(old_means, new_means, eps = 1e-6){

  om <- old_means %>% select(-label) %>% as.matrix()
  nm <- new_means %>% select(-label) %>% as.matrix()
  
  distances <- sqrt(rowSums((om - nm)^2))
  
  if (mean(distances) < eps) TRUE else FALSE
}

F) 

In [5]:
mykmeans <- function(data, n_clusters, eps = 1e-6, max_iter = 100){
  
  labels <- label_randomly(nrow(data), n_clusters)
  
  old_means <- get_cluster_means(data, labels)
  
  done <- FALSE
  iter <- 0

  while (!done && iter < max_iter){

    labels <- assign_cluster(data, old_means)

    new_means <- get_cluster_means(data, labels)
    
    if (kmeans_done(old_means, new_means, eps)){
      done <- TRUE
    }
    
    old_means <- new_means
    iter <- iter + 1
  }
  
  if (iter == max_iter && !done) {
    warning("K-Means did not converge within the maximum number of iterations.")
  }
  
  cat(paste("K-Means converged after", iter, "iterations.\n"))

  return(list(labels = labels, means = new_means))
}

QUESTION 2

A)

In [6]:
voltages_df <- read_csv("voltages_df.csv")

Rows: 900 Columns: 250
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (250): 0, 1.00401606425703, 2.00803212851406, 3.01204819277108, 4.016064...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


B)

In [7]:
n_clusters <- 3

print(paste("Running custom mykmeans with k =", n_clusters))
my_results <- mykmeans(voltages_df, n_clusters)

print("--- Custom mykmeans Results (Cluster Labels) ---")
print(my_results$labels)

print("--- Custom mykmeans Results (Cluster Means/Centroids) ---")
print(my_results$means)

[1] "Running custom mykmeans with k = 3"


ERROR: Error in label_randomly(nrow(data), n_clusters): could not find function "label_randomly"


C)

In [ ]:
n_clusters <- 3 

print("\n--- RESULTS FROM CUSTOM mykmeans (Part b) ---")
my_results <- mykmeans(voltages_df, n_clusters)


print("Cluster Labels (mykmeans, in $labels):")
print(my_results$labels)

print("Cluster Means (mykmeans, in $means):")
print(my_results$means)

voltages_matrix <- as.matrix(voltages_df)

print("\n--- RESULTS FROM R's BUILT-IN kmeans (Part c) ---")

r_results <- stats::kmeans(voltages_matrix, centers = n_clusters, nstart = 25) 

print("Cluster Labels (R's kmeans, in $cluster):")
print(r_results$cluster) 

print("Cluster Means (R's kmeans, in $centers):")
print(r_results$centers)

D) The raw lists of cluster labels are not identical because the label numbers (1, 2, 3) are arbitrary identifiers, meaning that a point labeled '1' in my function might be labeled '2' in R's function. However, when comparing the final cluster means, the centroids are effectively the same across both functions, indicating both algorithms converged to the same three physical locations in the data space. For instance, my custom kmeans cluster 1 values (-1.03,0.938) correspond with R's cluster 2 values (-1.031463,0.9381238). This result confirms that my custom implementation of the K-Means algorithm successfully mirrored the outcome of R's built-in function.

QUESTION 3

A)

When using a for loop, cluster assignment is done iteratively:
1. Loop through each data point one by one.
2. For each point, loop through all cluster centroids.
3. Compute the distance from the point to each centroid (usually squared Euclidean distance).
4. Determine which centroid is closest to the point.
5. Assign the point to that cluster.
6. Repeat for all points to complete one iteration of cluster reassignment.

This method is straightforward and mirrors the algorithm description directly, but it can be slow for large datasets, since it explicitly computes distances one point at a time.

B)

When using vectorization, explicit loops are eliminated by performing operations on entire arrays or matrices at once:

1. Represent the data as a matrix and the centroids as another matrix.
2. Compute the distances from all points to all centroids in one matrix operation, e.g., using matrix subtraction and row-wise sums.
3. For each point, find the index of the minimum distance across centroids.
4. Assign clusters based on these indices.

Vectorization leverages optimized internal code and avoids the overhead of repeated loops.

C)

Vectorized code is more efficient than using for loops because it performs operations on entire matrices at once, avoiding the overhead of repeatedly iterating through points. For loops are easier to read and understand but can become very slow for large datasets.

QUESTION 4

K-means can fail when the clusters in the data are not roughly spherical, have very different sizes, or have overlapping or irregular shapes. This failure happens because K-means assumes that each cluster is defined solely by its centroid, and that points belong to the cluster whose center is closest. Since K-means has no way of accounting for variance, standard deviation, or multiple clusters sharing the same center, its simple algorithm can produce misleading results in these situations, despite being very efficient under ideal assumptions.

QUESTION 5

Gaussian Mixture Models assume that the data are drawn from multiple Gaussian distributions, each with its own mean and covariance. This allows clusters to have different sizes, shapes, and orientations, unlike K-means which assumes spherical, equally sized clusters. The parameters of each Gaussian are estimated from the data using expectation-maximization, iteratively refining cluster assignments to best fit the distributions.

QUESTION 6

Spectral clustering assumes that points are more likely to belong to the same cluster if they are close or similar according to some metric. This minimal assumption allows it to work on data that do not reside in a vector space and even on purely relational data, such as graphs or adjacency matrices. By using the Graph Laplacian and its eigenvectors, spectral clustering creates a low-dimensional representation of the connectivity structure, which can then be clustered with standard methods like k-means, capturing complex cluster shapes that traditional methods might miss.

QUESTION 7

The gap statistic method is a standard approach for choosing the number of clusters in a dataset. It works by comparing the clustering results for each value of 
𝐾
K to a reference dataset where the points are randomized within the same domain. By measuring the difference in dispersion between the real and randomized data, we identify the “knee” in the gap, which suggests the most appropriate number of clusters.